In [ ]:
!pip3 uninstall -y tensorflow
!pip3 install tensorflow==1.15

In [1]:
import tensorflow as tf
tf.__version__

'1.15.0'

In [2]:
from google.colab import drive
drive.mount("/content/drive/")
!ls

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
drive  sample_data


In [3]:
cd drive/My Drive/cose474/assignment4/

/content/drive/My Drive/cose474/assignment4


In [6]:
import numpy as np
from tensorflow.python.training import moving_averages

##############################################################################################################
#                    TODO : X1 ~ X10에 올바른 숫자 또는 변수를 채워넣어 ResNet32 코드를 완성할 것                 #
##############################################################################################################


class ResNet:
    def __init__(self, config):

        self._num_residual_units = config.num_residual_units #unit 개수, 즉 n의 크기
        self._batch_size = config.batch_size
        self._relu_leakiness = config.relu_leakiness
        self._num_classes = config.num_classes # label 개수 (10개-airplane, automobile, bird, cat, deer, dog, frog, horse, ship, truck)
        self._l2_reg_lambda = config.l2_reg_lambda

        self.X = tf.placeholder(tf.float32, [None, 32, 32, 3], name="X")
        self.Y = tf.placeholder(tf.float32, [None, self._num_classes], name="Y")
        self.extra_train_ops = [] # batch norm 파라미터 학습을 위한 operation 저장할 변수

        filters = [16, 16, 32, 64] # 층별 filter 개수 (output channel), 16 -> 16 -> 32 -> 64, filter 개수 2배는 conv layer 에서 stride 2 지정할 때마다
        activate_before_residual = [True, False, False] # activation 연산 수행 시점의 residual 연산 전후 여부

        with tf.variable_scope('init'): # 최초 convolutional layer
          x = self._conv('init_conv', self.X, 3, 3, filters[0], strides=[1,1,1,1]) # 필터 사이즈는 3 고정, in-filters 수는 rgb input 이기에 3, output filters는 16, stride는 1로 고정(마지막 층만 stride 2)

        with tf.variable_scope('unit_1_0'):
          x = self._residual(x, filters[0], filters[1], activate_before_residual[0], strides=[1, 1, 1, 1]) #첫 번째 residual unit
        for i in range(1, self._num_residual_units): #나머지 residual unit (n이 3이라면 두번째, 세번째 unit들)
          with tf.variable_scope('unit_1_%d' % i):
            x = self._residual(x, filters[1], filters[1], strides=[1, 1, 1, 1])

        with tf.variable_scope('unit_2_0'):
          x = self._residual(x, filters[1], filters[2], activate_before_residual[1], strides=[1, 2, 2, 1])
        for i in range(1, self._num_residual_units):
          with tf.variable_scope('unit_2_%d' % i):
            x = self._residual(x, filters[2], filters[2], strides=[1, 1, 1, 1])

        with tf.variable_scope('unit_3_0'):
          x = self._residual(x, filters[2], filters[3], activate_before_residual[2], strides=[1, 2, 2, 1])
        for i in range(1, self._num_residual_units):
          with tf.variable_scope('unit_3_%d' % i):
            x = self._residual(x, filters[3], filters[3], strides=[1, 1, 1, 1])

        with tf.variable_scope('unit_last'):
          x = self._batch_norm('final_bn', x)
          x = self._relu(x, self._relu_leakiness)
          x = self._global_avg_pool(x) # (?, 8, 8, 64) 크기의 feature maps을 average pooling을 통해 (?, 8, 8)로 요약 및 축소

        with tf.variable_scope('logit'):
          logits = self._fully_connected(x, self._num_classes) # fully connected layer
          self.predictions = tf.nn.softmax(logits)
          self.predictions = tf.argmax(self.predictions, 1, name="predictions")

        with tf.variable_scope('loss'):
          xent = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=self.Y)
          self.loss = tf.reduce_mean(xent, name='xent')
          self.loss += self._decay()

        with tf.name_scope("accuracy"):
            correct_predictions = tf.equal(self.predictions, tf.argmax(self.Y, 1))
            self.accuracy = tf.reduce_mean(tf.cast(correct_predictions, "float"), name="accuracy")


    def _conv(self, name, x, filter_size, in_filters, out_filters, strides):
        with tf.variable_scope(name):
            n = filter_size * filter_size * out_filters # he 초기화를 위한 크기 계산
            # filter size 가로, 세로, input channel, output channel 크기로 filter 초기화
            kernel = tf.get_variable(
                'DW', [filter_size, filter_size, in_filters, out_filters],
                tf.float32, initializer=tf.random_normal_initializer(stddev=np.sqrt(2.0 / n)))
            return tf.nn.conv2d(x, kernel, strides, padding='SAME') # convolution 연산

    def _residual(self, x, in_filter, out_filter, activate_before_residual=False, strides=[1, 1, 1, 1]):

        if activate_before_residual:
            with tf.variable_scope('common_activation'):
                x = self._batch_norm('init_bn', x)
                x = self._relu(x, self._relu_leakiness)
                orig_x = x # skip connection 때 넘 겨줄 feature orig_x에 복사
        else:
            with tf.variable_scope('residual_activation'):
                orig_x = x # skip connection 때 넘 겨줄 feature orig_x에 복사
                x = self._batch_norm('init_bn', x)
                x = self._relu(x, self._relu_leakiness)

        with tf.variable_scope('sub1'):
            x = self._conv('conv1', x, 3, in_filter, out_filter, strides) #필터 사이즈는 3(모든 conv layer에서 동일한 filter size), strides는 지정된 stride 사용 2도 사용 가능

        with tf.variable_scope('sub2'):
            x = self._batch_norm('bn2', x)
            x = self._relu(x, self._relu_leakiness)
            x = self._conv('conv2', x, 3, out_filter, out_filter, strides = [1,1,1,1]) #필터 사이즈 3, strides는 무조건 1

        with tf.variable_scope('sub_add'):
            if in_filter != out_filter: # stride 크기가 2일 때 channel 크기가 안맞는 경우 크기 조정을 통해 skip connection이 원활하게 조정
                orig_x = tf.nn.avg_pool(orig_x, strides, strides, 'VALID') # pooling으로 feature map 크기 맞추고
                orig_x = tf.pad(
                    orig_x, [[0, 0], [0, 0], [0, 0],
                             [(out_filter - in_filter) // 2, (out_filter - in_filter) // 2]]) # padding으로 채널 맞춤
            x += orig_x # skip connection

        tf.logging.debug('image after unit %s', x.get_shape())
        return x

    def _relu(self, x, leakiness=0.0):
        return tf.where(tf.less(x, 0.0), leakiness * x, x, name='leaky_relu')

    def _batch_norm(self, name, x):
        with tf.variable_scope(name):
            params_shape = [x.get_shape()[-1]]

            beta = tf.get_variable(
                'beta', params_shape, tf.float32,
                initializer=tf.constant_initializer(0.0, tf.float32))
            gamma = tf.get_variable(
                'gamma', params_shape, tf.float32,
                initializer=tf.constant_initializer(1.0, tf.float32))

            mean, variance = tf.nn.moments(x, [0, 1, 2], name='moments')

            moving_mean = tf.get_variable(
                'moving_mean', params_shape, tf.float32,
                initializer=tf.constant_initializer(0.0, tf.float32),
                trainable=False)
            moving_variance = tf.get_variable(
                'moving_variance', params_shape, tf.float32,
                initializer=tf.constant_initializer(1.0, tf.float32),
                trainable=False)

            self.extra_train_ops.append(moving_averages.assign_moving_average(
                moving_mean, mean, 0.9))
            self.extra_train_ops.append(moving_averages.assign_moving_average(
                moving_variance, variance, 0.9))

            y = tf.nn.batch_normalization(x, mean, variance, beta, gamma, 0.001)
            y.set_shape(x.get_shape())
            return y

    def _fully_connected(self, x, out_dim):
        dim = tf.reduce_prod(x.get_shape()[1:]).eval()
        print(dim)
        x = tf.reshape(x, [-1, dim])
        w = tf.get_variable(
            'DW', [dim, out_dim],
            initializer=tf.uniform_unit_scaling_initializer(factor=1.0))
        b = tf.get_variable('biases', [out_dim],
                            initializer=tf.constant_initializer())
        return tf.nn.xw_plus_b(x, w, b)

    def _global_avg_pool(self, x):
        # (?, 8, 8, 64) 크기의 feature maps을 average pooling을 통해 (?, 1, 1, 64)로 요약 및 축소
        assert x.get_shape().ndims == 4
        return tf.reduce_mean(x, [1, 2])

    def _decay(self):
        """L2 weight decay loss."""
        costs = []
        for var in tf.trainable_variables(): # 학습 가능한 모든 변수들의
            if var.op.name.find(r'DW') > 0:
                costs.append(tf.nn.l2_loss(var)) # l2 norm을 구해서 costs에 저장

        return tf.multiply(self._l2_reg_lambda, tf.add_n(costs)) # costs안에 값들을 모두 더한 뒤 lambda 값에 곱해줌

In [4]:
tf.flags.DEFINE_string('f', '', 'kernel')
def del_all_flags(FLAGS):
  flags_dict = FLAGS._flags()
  keys_list = [keys for keys in flags_dict]
  for keys in keys_list:
    FLAGS.__delattr__(keys)

del_all_flags(tf.flags.FLAGS)

In [13]:
#del_all_flags(tf.flags.FLAGS)
#tf.flags.DEFINE_string('f', '', 'kernel')

In [5]:
import os
import time
import datetime
from tensorflow.keras.datasets.cifar10 import load_data
import data_helpers as dh

# Model Hyperparameters
tf.flags.DEFINE_float("lr", 0.15, "learning rate (default=0.1)")
tf.flags.DEFINE_float("lr_decay", 0.01, "learning rate decay rate(default=0.1)")
tf.flags.DEFINE_float("l2_reg_lambda", 0.0001, "L2 regularization lambda (default: 0.0)")
tf.flags.DEFINE_float("relu_leakiness", 0.1, "relu leakiness (default: 0.1)") # Setting 1~4: 0.1 
tf.flags.DEFINE_integer("num_residual_units", 3, "The number of residual_units (default: 5)")
tf.flags.DEFINE_integer("num_classes", 10, "The number of classes (default: 10)")

# Training parameters
tf.flags.DEFINE_integer("batch_size", 128, "Batch Size (default: 64)")
tf.flags.DEFINE_integer("num_epochs", 200, "Number of training epochs (default: 200)")
tf.flags.DEFINE_integer("evaluate_every", 352, "Evaluate model on dev set after this many steps (default: 100)")
tf.flags.DEFINE_integer("checkpoint_every", 352, "Save model after this many steps (default: 100)")
tf.flags.DEFINE_integer("num_checkpoints", 3, "Number of checkpoints to store (default: 5)")
tf.flags.DEFINE_boolean("data_augmentation", True, "data augmentation option")
tf.flags.DEFINE_string('f', '', 'kernel')

# Misc Parameters
tf.flags.DEFINE_boolean("allow_soft_placement", True, "Allow device soft device placement")
tf.flags.DEFINE_boolean("log_device_placement", False, "Log placement of ops on devices")
FLAGS = tf.flags.FLAGS

(x_train_val, y_train_val), (x_test, y_test) = load_data()
x_train, y_train, x_test, y_test, x_val, y_val = dh.shuffle_data(x_train_val, y_train_val, x_test, y_test, FLAGS.num_classes)

with tf.Graph().as_default():
    session_conf = tf.ConfigProto(
      allow_soft_placement=FLAGS.allow_soft_placement,
      log_device_placement=FLAGS.log_device_placement)
    sess = tf.Session(config=session_conf)
    with sess.as_default():
        resnet = ResNet(FLAGS) #ResNet 클래스의 인스턴스 생성 후 Hyperparameter가 정의돼 있는 FLAGS로 초기화

        # Define Training procedure
        global_step = tf.Variable(0, name="global_step", trainable=False)
        decayed_lr = tf.train.exponential_decay(FLAGS.lr, global_step, 3000, 1- FLAGS.lr_decay, staircase=True)
        optimizer = tf.train.MomentumOptimizer(learning_rate=decayed_lr, momentum=0.9)
        #optimizer = tf.train.AdamOptimizer(learning_rate=decayed_lr)
        grads_and_vars = optimizer.compute_gradients(resnet.loss)
        train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)
        train_ops = [train_op] + resnet.extra_train_ops
        train_ops = tf.group(*train_ops)

        # Output directory for models and summaries
        timestamp = str(int(time.time()))
        out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", timestamp))
        print("Writing to {}\n".format(out_dir))

        # Summaries for loss and accuracy
        loss_summary = tf.summary.scalar("loss", resnet.loss)
        acc_summary = tf.summary.scalar("accuracy", resnet.accuracy)

        # Train Summaries
        train_summary_op = tf.summary.merge([loss_summary, acc_summary])
        train_summary_dir = os.path.join(out_dir, "summaries", "train")
        train_summary_writer = tf.summary.FileWriter(train_summary_dir, sess.graph)

        # Dev summaries
        dev_summary_op = tf.summary.merge([loss_summary, acc_summary])
        dev_summary_dir = os.path.join(out_dir, "summaries", "dev")
        dev_summary_writer = tf.summary.FileWriter(dev_summary_dir, sess.graph)

        # Checkpoint directory. Tensorflow assumes this directory already exists so we need to create it
        checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))
        checkpoint_prefix = os.path.join(checkpoint_dir, "model")
        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)
        saver = tf.train.Saver(tf.global_variables(), max_to_keep=FLAGS.num_checkpoints)

        sess.run(tf.global_variables_initializer())

        def train_step(x_batch, y_batch):
            """
            A single training step
            """
            feed_dict = {
              resnet.X: x_batch,
              resnet.Y: y_batch
            }
            _, step, lr, summaries, loss, accuracy = sess.run(
                [train_ops, global_step, decayed_lr, train_summary_op, resnet.loss, resnet.accuracy],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, lr {}, loss {:g}, acc {:g}".format(time_str, step, lr, loss, accuracy))
            train_summary_writer.add_summary(summaries, step)

        def dev_step(x_batch, y_batch, writer=None):
            """
            Evaluates model on a dev set
            """
            feed_dict = {
              resnet.X: x_batch,
              resnet.Y: y_batch
            }
            step, summaries, loss, accuracy = sess.run(
                [global_step, dev_summary_op, resnet.loss, resnet.accuracy],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
            if writer:
                writer.add_summary(summaries, step)
            return accuracy

        # Generate batches
        if FLAGS.data_augmentation:  # data augmentation 적용시
            batches = dh.batch_iter_aug(x_train, y_train, FLAGS.batch_size, FLAGS.num_epochs)
        else:
            batches = dh.batch_iter(x_train, y_train, FLAGS.batch_size, FLAGS.num_epochs)
        # Training loop. For each batch...
        max = 0
        start_time = time.time()
        for batch in batches:
            x_batch, y_batch = zip(*batch)
            train_step(x_batch, y_batch)
            current_step = tf.train.global_step(sess, global_step)
            if current_step % FLAGS.evaluate_every == 0:
                print("\nEvaluation:")
                accuracy = dev_step(x_val, y_val, writer=dev_summary_writer)
                print("")
                if accuracy > max:
                    max = accuracy
                    path = saver.save(sess, checkpoint_prefix, global_step=current_step)
                    print("Saved model checkpoint to {}\n".format(path))
        training_time = (time.time() - start_time) / 60
        print('training time: ', training_time) # training time 추가


Streaming output truncated to the last 5000 lines.
2020-10-02T21:44:51.140772: step 65463, lr 0.12145921587944031, loss 0.437463, acc 0.90625
2020-10-02T21:44:51.280905: step 65464, lr 0.12145921587944031, loss 0.401565, acc 0.929688
2020-10-02T21:44:51.422254: step 65465, lr 0.12145921587944031, loss 0.362551, acc 0.929688
2020-10-02T21:44:51.566313: step 65466, lr 0.12145921587944031, loss 0.310865, acc 0.976562
2020-10-02T21:44:51.705525: step 65467, lr 0.12145921587944031, loss 0.329644, acc 0.945312
2020-10-02T21:44:51.849003: step 65468, lr 0.12145921587944031, loss 0.371635, acc 0.945312
2020-10-02T21:44:51.988890: step 65469, lr 0.12145921587944031, loss 0.291889, acc 0.945312
2020-10-02T21:44:52.128360: step 65470, lr 0.12145921587944031, loss 0.396644, acc 0.9375
2020-10-02T21:44:52.271717: step 65471, lr 0.12145921587944031, loss 0.324628, acc 0.945312
2020-10-02T21:44:52.354439: step 65472, lr 0.12145921587944031, loss 0.431507, acc 0.930556

Evaluation:
2020-10-02T21:44:53

- Setting 1: 1601446590
- Setting 2: 1601462332
- Setting 3: 1601481770
- Setting 4: 1601486008
- Setting 5: 1601665510

In [14]:
from tensorflow.keras.datasets.cifar10 import load_data

(x_train_val, y_train_val), (x_test, y_test) = load_data()



# Eval Parameters
tf.flags.DEFINE_string("checkpoint_dir", "./runs/1601665510/checkpoints", "Checkpoint directory from training run")

FLAGS = tf.flags.FLAGS
# ==================================================
y_test_one_hot = np.eye(10)[y_test]
y_test_one_hot = np.squeeze(y_test_one_hot, axis=1)
x_test = (x_test/127.5) - 1

checkpoint_file = tf.train.latest_checkpoint(FLAGS.checkpoint_dir)
graph = tf.Graph()
with graph.as_default():
    sess = tf.Session()
    with sess.as_default():
        saver = tf.train.import_meta_graph("{}.meta".format(checkpoint_file))
        saver.restore(sess, checkpoint_file)

        # Get the placeholders from the graph by name
        X = graph.get_operation_by_name("X").outputs[0]
        Y = graph.get_operation_by_name("Y").outputs[0]
        accuracy = graph.get_operation_by_name("accuracy/accuracy").outputs[0]

        test_accuracy = sess.run(accuracy, feed_dict={X: x_test, Y: y_test_one_hot})
        print('Test Max Accuracy:', test_accuracy)

INFO:tensorflow:Restoring parameters from /content/drive/My Drive/cose474/assignment4/runs/1601665510/checkpoints/model-55616
Test Max Accuracy: 0.8878
